<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/10170/logos/header.png" width="1000px">

**This notebook is an exercise in the [Machine Learning Explainability](https://www.kaggle.com/learn/machine-learning-explainability) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/permutation-importance).**

---


## Intro

You will think about and calculate permutation importance with a sample of data from the [Taxi Fare Prediction](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction) competition.

We won't focus on data exploration or model building for now. You can just run the cell below to 
- Load the data
- Divide the data into training and validation
- Build a model that predicts taxi fares
- Print a few rows for you to review

In [ ]:
# Loading data, dividing, modeling and EDA below
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv', nrows=50000)
data.sample(10)

In [ ]:
# Remove data with extreme outlier coordinates or negative fares
data = data.query('pickup_latitude > 40.7 and pickup_latitude < 40.8 and ' +
                  'dropoff_latitude > 40.7 and dropoff_latitude < 40.8 and ' +
                  'pickup_longitude > -74 and pickup_longitude < -73.9 and ' +
                  'dropoff_longitude > -74 and dropoff_longitude < -73.9 and ' +
                  'fare_amount > 0'
                  )

In [ ]:
#Target variable
y = data.fare_amount

In [ ]:
base_features = ['pickup_longitude',
                 'pickup_latitude',
                 'dropoff_longitude',
                 'dropoff_latitude',
                 'passenger_count']

X = data[base_features]
X.sample(10)

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [ ]:
print(X.size, y.size, train_X.size/X.size, val_X.size/X.size, train_y.size/y.size, val_y.size/y.size)

In [ ]:
first_model = RandomForestRegressor(n_estimators=50, random_state=1).fit(train_X, train_y)
first_model

In [ ]:
first_model_tree = first_model.estimators_[0]
first_model_tree

In [ ]:
first_model_tree.n_features_

# Visualize a Decision Tree in 4 Ways
# Intro to ML - Underfitting and Overfitting
# https://www.kaggle.com/georgezoto/intro-to-ml-underfitting-and-overfitting

In [ ]:
from sklearn import tree
#from matplotlib import pyplot as plt

In [ ]:
text_representation = tree.export_text(first_model_tree)

#The text representation of this tree is pretty long so let's just show a few nodes instead
print(text_representation[:5000])

In [ ]:
#This will take a couple of minutes and use all your CPU
#fig = plt.figure(figsize=(25,20))
#_ = tree.plot_tree(first_model_tree, feature_names=base_features, filled=True)

In [ ]:
#import graphviz
#print(graphviz.__version__)

In [ ]:
#Scroll up and down, left and right to navigate this good looking tree :) 
#graphviz_tree = tree.export_graphviz(first_model_tree, out_file=None, 
#                                feature_names=base_features,
#                                filled=True)
#graphviz.Source(graphviz_tree, format="png") 

In [ ]:
from sklearn import tree
import graphviz

tree_graph = tree.export_graphviz(first_model_tree, out_file=None, feature_names=base_features)
graphviz.Source(tree_graph)

In [ ]:
# Environment Set-Up for feedback system.
from learntools.core import binder
binder.bind(globals())
from learntools.ml_explainability.ex2 import *
print("Setup Complete")

# show data
print("Data sample:")
data.head()

The following two cells may also be useful to understand the values in the training data:

In [ ]:
train_X.describe()

In [ ]:
train_y.describe()

## Question 1

The first model uses the following features
- pickup_longitude
- pickup_latitude
- dropoff_longitude
- dropoff_latitude
- passenger_count

Before running any code... which variables seem potentially useful for predicting taxi fares? Do you think permutation importance will necessarily identify these features as important?

Once you've thought about it, run `q_1.solution()` below to see how you might think about this before running the code.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_1.solution()

## Question 2

Create a `PermutationImportance` object called `perm` to show the importances from `first_model`.  Fit it with the appropriate data and show the weights.

For your convenience, the code from the tutorial has been copied into a comment in this code cell.

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

# Make a small change to the code below to use in this problem. 
perm = PermutationImportance(first_model, random_state=0).fit(val_X, val_y) #random_state=1 Incorrect???

In [ ]:
# Check your answer
q_2.check()

In [ ]:
# uncomment the following line to visualize your results
eli5.show_weights(perm, feature_names = val_X.columns.tolist())

<img src="https://upload.wikimedia.org/wikipedia/commons/e/ef/FedStats_Lat_long.svg" width="400px">

# Another way to visualize feature importance
# Intro to ML - Random Forests
# https://www.kaggle.com/georgezoto/intro-to-ml-random-forests

In [ ]:
from matplotlib import pyplot as plt
from sklearn.inspection import permutation_importance

# the permutation based importance
perm_importance = permutation_importance(first_model, val_X, val_y)

sorted_idx = perm_importance.importances_mean.argsort()
#print(features, sorted_idx)
#plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")
plt.barh([x for _,x in sorted(zip(perm_importance.importances_mean, val_X.columns.tolist()))], perm_importance.importances_mean[sorted_idx]);

# Let's visualize a few points on the map

In [ ]:
import folium
from folium import Choropleth, Circle, Marker

In [ ]:
df=data.copy()

locations = df[['pickup_latitude', 'pickup_longitude']].dropna()
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

In [ ]:
len(df)

In [ ]:
# Create a base map
map = folium.Map(location=locationlist[7], zoom_start=13)


def color_producer(val):
    if val <= train_y.mean():
        return 'forestgreen'
    else:
        return 'darkred'

# Add a bubble map to the base map
for i in range(0,len(df[:1000])):
    Circle(
        location=[df.iloc[i]['pickup_latitude'], df.iloc[i]['pickup_longitude']],
        radius=df.iloc[i]['fare_amount']*10,
        tooltip=df.iloc[i]['fare_amount'],
        color=color_producer(df.iloc[i]['fare_amount'])).add_to(map)

# Display the map
map

Uncomment the lines below for a hint or to see the solution.

In [ ]:
#q_2.hint()
#q_2.solution()

## Question 3
Before seeing these results, we might have expected each of the 4 directional features to be equally important.

But, on average, the latitude features matter more than the longititude features. Can you come up with any hypotheses for this?

After you've thought about it, check here for some possible explanations:

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_3.solution()

## Question 4

Without detailed knowledge of New York City, it's difficult to rule out most hypotheses about why latitude features matter more than longitude.

A good next step is to disentangle the effect of being in certain parts of the city from the effect of total distance traveled.  

The code below creates new features for longitudinal and latitudinal distance. It then builds a model that adds these new features to those you already had.

Fill in two lines of code to calculate and show the importance weights with this new set of features. As usual, you can uncomment lines below to check your code, see a hint or get the solution.

In [ ]:
# create new features
data['abs_lon_change'] = abs(data.dropoff_longitude - data.pickup_longitude)
data['abs_lat_change'] = abs(data.dropoff_latitude - data.pickup_latitude)

features_2  = ['pickup_longitude',
               'pickup_latitude',
               'dropoff_longitude',
               'dropoff_latitude',
               'abs_lat_change',
               'abs_lon_change']

X = data[features_2]
new_train_X, new_val_X, new_train_y, new_val_y = train_test_split(X, y, random_state=1)
second_model = RandomForestRegressor(n_estimators=30, random_state=1).fit(new_train_X, new_train_y)

# Create a PermutationImportance object on second_model and fit it to new_val_X and new_val_y
# Use a random_state of 1 for reproducible results that match the expected solution.
perm2 = PermutationImportance(second_model, random_state=8).fit(new_val_X, new_val_y) # random_state=1 Incorrect ???

# show the weights for the permutation importance you just calculated
eli5.show_weights(perm2, feature_names = features_2)

In [ ]:
data.describe()

In [ ]:
# Check your answer
q_4.check()

How would you interpret these importance scores? Distance traveled seems far more important than any location effects. 

But the location still affects model predictions, and dropoff location now matters slightly more than pickup location. Do you have any hypotheses for why this might be? The techniques in the next lessons will help you dive into this more.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_4.solution()

## Question 5

A colleague observes that the values for `abs_lon_change` and `abs_lat_change` are pretty small (all values are between -0.1 and 0.1), whereas other variables have larger values.  Do you think this could explain why those coordinates had larger permutation importance values in this case?  

Consider an alternative where you created and used a feature that was 100X as large for these features, and used that larger feature for training and importance calculations. Would this change the outputted permutaiton importance values?

Why or why not?

After you have thought about your answer, either try this experiment or look up the answer in the cell below.

In [ ]:
# create new features
data['abs_lon_change'] = abs(data.dropoff_longitude - data.pickup_longitude)*1000
data['abs_lat_change'] = abs(data.dropoff_latitude - data.pickup_latitude)*1000

features_2  = ['pickup_longitude',
               'pickup_latitude',
               'dropoff_longitude',
               'dropoff_latitude',
               'abs_lat_change',
               'abs_lon_change']

X = data[features_2]
new_train_X, new_val_X, new_train_y, new_val_y = train_test_split(X, y, random_state=1)
second_model = RandomForestRegressor(n_estimators=30, random_state=1).fit(new_train_X, new_train_y)

# Create a PermutationImportance object on second_model and fit it to new_val_X and new_val_y
# Use a random_state of 1 for reproducible results that match the expected solution.
perm2 = PermutationImportance(second_model, random_state=1).fit(new_val_X, new_val_y) 

# show the weights for the permutation importance you just calculated
eli5.show_weights(perm2, feature_names = features_2)

In [ ]:
data.describe()

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_5.solution()

## Question 6

You've seen that the feature importance for latitudinal distance is greater than the importance of longitudinal distance. From this, can we conclude whether travelling a fixed latitudinal distance tends to be more expensive than traveling the same longitudinal distance?

Why or why not? Check your answer below.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_6.solution()

## Keep Going

Permutation importance is useful for debugging, understanding your model, and communicating a high-level overview from your model.  

Next, learn about **[partial dependence plots](https://www.kaggle.com/dansbecker/partial-plots)** to see **how** each feature affects predictions.


---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161307) to chat with other Learners.*